# The Miller-Tucker-Zemlin formulation for ATSP

<b>Goal:</b> Implement the MTZ formulation and solve ATSP on a set of locations in Zurich.

The Miller-Tucker-Zemlin formulation is the following IP formulation for the ATSP problem:
$$
\begin{array}{rrcll}
\min & \displaystyle \sum_{i\in [n]}\sum_{j\in [n]\setminus\{ i\} }d_{ij}x_{ij} \\
& \displaystyle \sum_{i\in [n]\setminus\{ j\} }x_{ij} & = & 1 & \forall j\in [n] \\
& \displaystyle \sum_{j\in [n]\setminus\{ i\} }x_{ij} & = & 1 & \forall i\in [n] \\
& u_{i}-u_{j}+nx_{ij} & \le & n-1 & \forall i, j\in {2,\ldots,n}, \ i\neq j\\
& x_{ij} & \in & \{ 0,1\} & \forall i, j\in [n], \ i\neq j \\
& u_{i} & \in &\{1,\ldots,n-1\} & \forall i\in{2,\ldots, n}
\end{array}
$$

## Required packages

To run the map visualization function, you need to install the `mplleaflet` package by running `conda install -c conda-forge mplleaflet`. Moreover, you need to make sure that the version of the `matplotlib` package is 3.3.2. Run `conda list` to see the list of installed packages and find `matplotlib` in the listing. If the version is different from 3.3.2, note it down in case you want to restore it later and then run `conda install matplotlib=3.3.2`. If you want to restore `matplotlib` to the original version, run the same command and replace 3.3.2 with the version number you noted down.

When running our visualization function, you may see a red box with a deprecation warning. It can be safely ignored, and the code should run just fine.

## Locations and distances

We selected a list of 21 points of interest in Zurich and determined biking distances between each pair of locations using OpenStreetMap. Note that the distances are already given to you, so you <b>do not</b> need to implement this step. Nevertheless, if you are interested, feel free to analyse and play with our implementation for querying the distances.

In [1]:
def get_distance_dict(coord_dict, base_distance_dict=None):
    '''Creates a dict of distances between all pairs of locations by requesting data from Open Street Map.
    
    Args:
        coord_dict: A dict keyed by locations containing their coordinates as (longitude, latitude) tuples.
        base_distance_dict: A dict of dicts keyed by locations containing distances between the locations
            which should be kept as-is. Defaults to None.
    
    Returns:
        A dict of dicts keyed by locations contatining all pairwise distances between the locations.
        Distances contained in base_distance_dict are kept as is, while the rest are obtained via
        querying OpenStreetMap API.
    '''
    import requests
    
    # Initialize distance_dict
    if base_distance_dict is None:
        base_distance_dict = {}
    distance_dict = {i: {j: base_distance_dict[i][j]
                         for j in base_distance_dict[i]} for i in base_distance_dict}
    
    for i in coord_dict:
        if i not in distance_dict:
            distance_dict[i] = {}
        
        for j in coord_dict:
            if i == j or base_distance_dict.get(i, {}).get(j, None) is not None:
                # Ignore pairs with identical locations and pairs contained in base_distance_dict
                continue
            
            # Send an HTTP request to OpenStreetMap
            print(f'Querying distance from {i} to {j}...')           
            r = requests.get('http://router.project-osrm.org/route/v1/bike/' +
                             f'{coord_dict[i][0]},{coord_dict[i][1]};{coord_dict[j][0]},{coord_dict[j][1]}')
            
            # Check for errors
            if r.status_code != 200:
                print(f'Error: request failed for distance from {i} to {j}.')
                print(r.text)
                continue
                
            # Extract the distance from the response
            json = r.json()
            distance_dict[i][j] = json['routes'][0]['distance']
            print(f'Distance: {distance_dict[i][j]}.')
            
    return distance_dict

In [2]:
coord_dict = {
    'ETH Zentrum': (8.547624336882576, 47.37644697416291),
    'ETH Hönggerberg': (8.5061934926737, 47.40870062362097),
    'Zurich HB Main Station': (8.540305577276873, 47.37799547669308),
    'Grossmünster': (8.54401108316836, 47.370282000357655),
    'Fraumünster': (8.541534659892758, 47.369893402956095),
    'Opera House': (8.546158788243805, 47.36519193610965),
    'Lindenhof': (8.541045383596446, 47.3730625314029),
    'FIFA Museum': (8.531580098513132, 47.363861186611594),
    'Landesmuseum': (8.540567362834716, 47.37924012074394),
    'Uetliberg': (8.492115949385951, 47.34985018763708),
    'Kunsthaus': (8.54822864559483, 47.37050503484518),
    'Zoo': (8.57448265597887, 47.387430999161886),
    'Anthropological Museum': (8.54729191389709, 47.398048586640634),
    'Botanical Garden': (8.560213827323311, 47.359500360644915),
    'Old Botanical Garden': (8.533749246305383, 47.371339427516304),
    'Irchel Park': (8.544148435043429, 47.39833822128007),
    'Letzigrund Stadium': (8.50379118408987, 47.38280610777361),
    'Käferberg': (8.520324422037788, 47.406710159031725),
    'Arboretum': (8.536225955023001, 47.36283216296779),
    'Heureka': (8.55233155323775, 47.353204022659526),
    'China Garden': (8.552054921784007, 47.35496093660575)
}
distance_dict = {
    'ETH Zentrum': {'ETH Hönggerberg': 6121.5, 'Zurich HB Main Station': 1719.8, 'Grossmünster': 1600, 'Fraumünster': 1713.9, 'Opera House': 2148.5, 'Lindenhof': 1891, 'FIFA Museum': 2440.9, 'Landesmuseum': 1210.6, 'Uetliberg': 7882.5, 'Kunsthaus': 960.8, 'Zoo': 3706.9, 'Anthropological Museum': 2751, 'Botanical Garden': 2537, 'Old Botanical Garden': 2185.5, 'Irchel Park': 3398.5, 'Letzigrund Stadium': 4546.6, 'Käferberg': 6685.9, 'Arboretum': 2244.2, 'Heureka': 3336.7, 'China Garden': 3084.3},
    'ETH Hönggerberg': {'ETH Zentrum': 6005.6, 'Zurich HB Main Station': 6281.6, 'Grossmünster': 7443.5, 'Fraumünster': 7557.4, 'Opera House': 8000.3, 'Lindenhof': 6639.8, 'FIFA Museum': 8000.4, 'Landesmuseum': 5772.4, 'Uetliberg': 12797, 'Kunsthaus': 6654.4, 'Zoo': 8260.2, 'Anthropological Museum': 4584.8, 'Botanical Garden': 8388.8, 'Old Botanical Garden': 6945.3, 'Irchel Park': 5266, 'Letzigrund Stadium': 6361.7, 'Käferberg': 3924.4, 'Arboretum': 8087.7, 'Heureka': 9180.1, 'China Garden': 8927.7},
    'Zurich HB Main Station': {'ETH Zentrum': 1660, 'ETH Hönggerberg': 5864.7, 'Grossmünster': 2124.1, 'Fraumünster': 2244.4, 'Opera House': 2680.9, 'Lindenhof': 997.5, 'FIFA Museum': 2358.1, 'Landesmuseum': 137.5, 'Uetliberg': 7076.9, 'Kunsthaus': 1335, 'Zoo': 5188.5, 'Anthropological Museum': 4200.5, 'Botanical Garden': 3069.5, 'Old Botanical Garden': 1303, 'Irchel Park': 4741.8, 'Letzigrund Stadium': 3741, 'Käferberg': 7295.3, 'Arboretum': 2638.4, 'Heureka': 4048.3, 'China Garden': 3795.9},
    'Grossmünster': {'ETH Zentrum': 1545.2, 'ETH Hönggerberg': 6712.3, 'Zurich HB Main Station': 1654.4, 'Fraumünster': 119.9, 'Opera House': 953.1, 'Lindenhof': 1079.9, 'FIFA Museum': 1434.8, 'Landesmuseum': 1145.2, 'Uetliberg': 7355.6, 'Kunsthaus': 1218.4, 'Zoo': 4973.2, 'Anthropological Museum': 3341.7, 'Botanical Garden': 2304.4, 'Old Botanical Garden': 1167.7, 'Irchel Park': 4928.6, 'Letzigrund Stadium': 3997.3, 'Käferberg': 7276.6, 'Arboretum': 1238.2, 'Heureka': 2071.7, 'China Garden': 1819.3},
    'Fraumünster': {'ETH Zentrum': 1995, 'ETH Hönggerberg': 7889, 'Zurich HB Main Station': 2815.9, 'Grossmünster': 119.9, 'Opera House': 1067.1, 'Lindenhof': 1976.5, 'FIFA Museum': 1296.3, 'Landesmuseum': 2306.7, 'Uetliberg': 6967.4, 'Kunsthaus': 1436.3, 'Zoo': 5087.1, 'Anthropological Museum': 4423.5, 'Botanical Garden': 2418.3, 'Old Botanical Garden': 1047.8, 'Irchel Park': 5071, 'Letzigrund Stadium': 3908.3, 'Käferberg': 8453.4, 'Arboretum': 1099.7, 'Heureka': 2185.6, 'China Garden': 1933.2},
    'Opera House': {'ETH Zentrum': 1939.9, 'ETH Hönggerberg': 7834, 'Zurich HB Main Station': 2877.7, 'Grossmünster': 1006, 'Fraumünster': 1149.9, 'Lindenhof': 2459.9, 'FIFA Museum': 1708.2, 'Landesmuseum': 2368.5, 'Uetliberg': 7629, 'Kunsthaus': 1381.2, 'Zoo': 5032, 'Anthropological Museum': 4368.5, 'Botanical Garden': 1683, 'Old Botanical Garden': 1711.9, 'Irchel Park': 5015.9, 'Letzigrund Stadium': 4572.4, 'Käferberg': 8398.3, 'Arboretum': 1511.6, 'Heureka': 1708.6, 'China Garden': 1456.2},
    'Lindenhof': {'ETH Zentrum': 1460.3, 'ETH Hönggerberg': 6364.6, 'Zurich HB Main Station': 1166.6, 'Grossmünster': 1718.6, 'Fraumünster': 1824.2, 'Opera House': 2275.4, 'FIFA Museum': 2096.8, 'Landesmuseum': 657.3, 'Uetliberg': 6815.6, 'Kunsthaus': 929.5, 'Zoo': 4783, 'Anthropological Museum': 3929.1, 'Botanical Garden': 2663.9, 'Old Botanical Garden': 882.9, 'Irchel Park': 4470.5, 'Letzigrund Stadium': 3494.7, 'Käferberg': 6929, 'Arboretum': 2211.2, 'Heureka': 3628.2, 'China Garden': 3375.8},
    'FIFA Museum': {'ETH Zentrum': 2609.6, 'ETH Hönggerberg': 7927.1, 'Zurich HB Main Station': 3255.1, 'Grossmünster': 1389.5, 'Fraumünster': 1269.6, 'Opera House': 1681.7, 'Lindenhof': 2415.7, 'Landesmuseum': 2745.9, 'Uetliberg': 6042, 'Kunsthaus': 2051, 'Zoo': 5701.7, 'Anthropological Museum': 5038.2, 'Botanical Garden': 3033, 'Old Botanical Garden': 1659.2, 'Irchel Park': 6559.1, 'Letzigrund Stadium': 3207.1, 'Käferberg': 8491.5, 'Arboretum': 385.6, 'Heureka': 2800.3, 'China Garden': 2547.9},
    'Landesmuseum': {'ETH Zentrum': 2169.3, 'ETH Hönggerberg': 5727.2, 'Zurich HB Main Station': 509.2, 'Grossmünster': 2633.3, 'Fraumünster': 2753.6, 'Opera House': 3190.2, 'Lindenhof': 1506.8, 'FIFA Museum': 2867.4, 'Uetliberg': 7586.1, 'Kunsthaus': 1844.2, 'Zoo': 5697.8, 'Anthropological Museum': 4276.2, 'Botanical Garden': 3578.7, 'Old Botanical Garden': 1812.3, 'Irchel Park': 4817.6, 'Letzigrund Stadium': 4114.8, 'Käferberg': 7157.7, 'Arboretum': 3147.7, 'Heureka': 4557.5, 'China Garden': 4305.1},
    'Uetliberg': {'ETH Zentrum': 16050.7, 'ETH Hönggerberg': 21285, 'Zurich HB Main Station': 15892.4, 'Grossmünster': 14352.9, 'Fraumünster': 14233, 'Opera House': 14960.6, 'Lindenhof': 15053, 'FIFA Museum': 13218.1, 'Landesmuseum': 15383.2, 'Kunsthaus': 15655.3, 'Zoo': 18980.6, 'Anthropological Museum': 18655, 'Botanical Garden': 16311.9, 'Old Botanical Garden': 14296.5, 'Irchel Park': 19196.4, 'Letzigrund Stadium': 15531.4, 'Käferberg': 24560, 'Arboretum': 13664.5, 'Heureka': 16079.2, 'China Garden': 15826.8},
    'Kunsthaus': {'ETH Zentrum': 841.4, 'ETH Hönggerberg': 6716.2, 'Zurich HB Main Station': 1759.9, 'Grossmünster': 797.5, 'Fraumünster': 911.4, 'Opera House': 1345.9, 'Lindenhof': 1342.1, 'FIFA Museum': 1638.3, 'Landesmuseum': 1250.7, 'Uetliberg': 7559.1, 'Zoo': 3933.5, 'Anthropological Museum': 3269.9, 'Botanical Garden': 1734.5, 'Old Botanical Garden': 1642, 'Irchel Park': 4932.5, 'Letzigrund Stadium': 4259.5, 'Käferberg': 7280.6, 'Arboretum': 1441.7, 'Heureka': 2534.1, 'China Garden': 2281.7},
    'Zoo': {'ETH Zentrum': 3604.4, 'ETH Hönggerberg': 8376, 'Zurich HB Main Station': 5120.6, 'Grossmünster': 4659.8, 'Fraumünster': 4773.7, 'Opera House': 5208.2, 'Lindenhof': 5244.6, 'FIFA Museum': 5500.6, 'Landesmuseum': 4611.4, 'Uetliberg': 11421.4, 'Kunsthaus': 3982.4, 'Anthropological Museum': 4624.7, 'Botanical Garden': 4835, 'Old Botanical Garden': 5504.2, 'Irchel Park': 5272.2, 'Letzigrund Stadium': 7947.4, 'Käferberg': 8940.4, 'Arboretum': 5303.9, 'Heureka': 5700.5, 'China Garden': 5448.1},
    'Anthropological Museum': {'ETH Zentrum': 3194, 'ETH Hönggerberg': 4736.1, 'Zurich HB Main Station': 3995, 'Grossmünster': 4635, 'Fraumünster': 4749, 'Opera House': 5183.5, 'Lindenhof': 4353.1, 'FIFA Museum': 5475.9, 'Landesmuseum': 3485.7, 'Uetliberg': 10432.5, 'Kunsthaus': 3957.6, 'Zoo': 5235.5, 'Botanical Garden': 5572, 'Old Botanical Garden': 4658.6, 'Irchel Park': 1295.4, 'Letzigrund Stadium': 4711.2, 'Käferberg': 5300.5, 'Arboretum': 5279.2, 'Heureka': 6371.7, 'China Garden': 6119.3},
    'Botanical Garden': {'ETH Zentrum': 2490.2, 'ETH Hönggerberg': 8365, 'Zurich HB Main Station': 3408.8, 'Grossmünster': 2429.7, 'Fraumünster': 2573.6, 'Opera House': 1822.6, 'Lindenhof': 2991, 'FIFA Museum': 3131.9, 'Landesmuseum': 2899.6, 'Uetliberg': 9052.7, 'Kunsthaus': 1648.9, 'Zoo': 5072.1, 'Anthropological Museum': 4918.8, 'Old Botanical Garden': 3135.5, 'Irchel Park': 6581.4, 'Letzigrund Stadium': 5908.3, 'Käferberg': 8929.4, 'Arboretum': 2935.2, 'Heureka': 1193.7, 'China Garden': 941.3},
    'Old Botanical Garden': {'ETH Zentrum': 2150.9, 'ETH Hönggerberg': 7055.2, 'Zurich HB Main Station': 1857.2, 'Grossmünster': 1150.3, 'Fraumünster': 1030.3, 'Opera House': 1715.7, 'Lindenhof': 1017.7, 'FIFA Museum': 1574.6, 'Landesmuseum': 1347.9, 'Uetliberg': 6293.4, 'Kunsthaus': 1620.1, 'Zoo': 5473.6, 'Anthropological Museum': 4619.7, 'Botanical Garden': 3067, 'Irchel Park': 5161.1, 'Letzigrund Stadium': 3325.1, 'Käferberg': 7619.6, 'Arboretum': 1417.3, 'Heureka': 2834.3, 'China Garden': 2581.9},
    'Irchel Park': {'ETH Zentrum': 3687.9, 'ETH Hönggerberg': 6640.9, 'Zurich HB Main Station': 3230, 'Grossmünster': 4478.6, 'Fraumünster': 4592.5, 'Opera House': 5035.4, 'Lindenhof': 3588.2, 'FIFA Museum': 4948.8, 'Landesmuseum': 2720.8, 'Uetliberg': 9667.6, 'Kunsthaus': 3689.5, 'Zoo': 6728.9, 'Anthropological Museum': 4454.6, 'Botanical Garden': 5423.9, 'Old Botanical Garden': 3893.7, 'Letzigrund Stadium': 5100, 'Käferberg': 7205.3, 'Arboretum': 5122.7, 'Heureka': 6215.2, 'China Garden': 5962.8},
    'Letzigrund Stadium': {'ETH Zentrum': 4665.5, 'ETH Hönggerberg': 5882.2, 'Zurich HB Main Station': 4615.1, 'Grossmünster': 4205.2, 'Fraumünster': 4085.3, 'Opera House': 4852.6, 'Lindenhof': 4211.3, 'FIFA Museum': 3276.3, 'Landesmuseum': 3934.1, 'Uetliberg': 7509.7, 'Kunsthaus': 4813.7, 'Zoo': 8257.6, 'Anthropological Museum': 5034.4, 'Botanical Garden': 6203.9, 'Old Botanical Garden': 3454.8, 'Irchel Park': 5713, 'Käferberg': 7687.8, 'Arboretum': 3556.5, 'Heureka': 5971.2, 'China Garden': 5718.8},
    'Käferberg': {'ETH Zentrum': 6460.6, 'ETH Hönggerberg': 3815, 'Zurich HB Main Station': 6736.6, 'Grossmünster': 7898.5, 'Fraumünster': 8012.4, 'Opera House': 8455.3, 'Lindenhof': 7094.8, 'FIFA Museum': 8455.4, 'Landesmuseum': 6227.4, 'Uetliberg': 13252, 'Kunsthaus': 7109.4, 'Zoo': 8715.2, 'Anthropological Museum': 5039.8, 'Botanical Garden': 8843.9, 'Old Botanical Garden': 7400.3, 'Irchel Park': 5721, 'Letzigrund Stadium': 6816.7, 'Arboretum': 8542.7, 'Heureka': 9635.1, 'China Garden': 9382.7},
    'Arboretum': {'ETH Zentrum': 2407.4, 'ETH Hönggerberg': 8301.4, 'Zurich HB Main Station': 3024.6, 'Grossmünster': 1187.2, 'Fraumünster': 1067.3, 'Opera House': 1479.4, 'Lindenhof': 2185.2, 'FIFA Museum': 439.3, 'Landesmuseum': 2515.4, 'Uetliberg': 6311.6, 'Kunsthaus': 1848.7, 'Zoo': 5499.5, 'Anthropological Museum': 4835.9, 'Botanical Garden': 2830.7, 'Old Botanical Garden': 1256.5, 'Irchel Park': 5483.4, 'Letzigrund Stadium': 3476.6, 'Käferberg': 8865.8, 'Heureka': 2598, 'China Garden': 2345.6},
    'Heureka': {'ETH Zentrum': 3186, 'ETH Hönggerberg': 9080, 'Zurich HB Main Station': 4123.8, 'Grossmünster': 2252.1, 'Fraumünster': 2396, 'Opera House': 1770, 'Lindenhof': 3706, 'FIFA Museum': 2954.3, 'Landesmuseum': 3614.6, 'Uetliberg': 8875.1, 'Kunsthaus': 2627.3, 'Zoo': 6278.1, 'Anthropological Museum': 5614.5, 'Botanical Garden': 1321.2, 'Old Botanical Garden': 2957.9, 'Irchel Park': 6262, 'Letzigrund Stadium': 5818.5, 'Käferberg': 9644.4, 'Arboretum': 2757.6, 'China Garden': 379.9},
    'China Garden': {'ETH Zentrum': 2806.1, 'ETH Hönggerberg': 8700.1, 'Zurich HB Main Station': 3743.9, 'Grossmünster': 1872.2, 'Fraumünster': 2016.1, 'Opera House': 1390.1, 'Lindenhof': 3326.1, 'FIFA Museum': 2574.4, 'Landesmuseum': 3234.7, 'Uetliberg': 8495.2, 'Kunsthaus': 2247.4, 'Zoo': 5898.2, 'Anthropological Museum': 5234.6, 'Botanical Garden': 941.3, 'Old Botanical Garden': 2578, 'Irchel Park': 5882.1, 'Letzigrund Stadium': 5438.6, 'Käferberg': 9264.5, 'Arboretum': 2377.7, 'Heureka': 252.4}
}
distance_dict = get_distance_dict(coord_dict, distance_dict)
location_list = list(coord_dict)

## Plotting the tour on the map

Below we provide you with the functionality for drawing an ATSP tour on a map. Again, you <b>do not</b> need to implement anything here. Nevertheless, if you are interested, feel free to analyse and play with our implementation.

In [4]:
import matplotlib.pyplot as plt
import mplleaflet
import networkx as nx

In [5]:
def visualize_mplleaflet_map(tour, coord_dict, filename=None, style='cartodb_positron'):
    '''Visualizes an ATSP tour as a networkx graph on an mplleaflet map and saves the file to the provided path.
    
    Args:
        tour: Tour represented as a list of locations in the input location_list in optimal traversal order.
              To represent a closed tour, the starting location should be repeated as the last location in the list.
        coord_dict: A dict keyed by locations containing their coordinates as (longitude, latitude) tuples.
        filename: A filename used to save the map. If None, then the map is not saved. Defaults to None.
        style: A string indicating the map tiles to use. Defaults to 'cartodb_positron'.
    '''
    fig, ax = plt.subplots()

    # Draw the graph corresponding to the ATSP tour
    G = nx.Graph()
    G.add_edges_from((tour[i - 1], tour[i]) for i in range(len(tour)))
    nx.draw_networkx(G, coord_dict, edge_color='red', with_labels=True)
    
    # Show the graph on the map
    if filename is not None:
        mplleaflet.show(fig=ax.figure, path=filename, tiles=style)
    else:
        mplleaflet.show(fig=ax.figure, tiles=style)

## Implementation of the MTZ formulation

In [6]:
from mip import *

In [ ]:
def atsp_via_mtz(location_list, distance_dict):
    '''Solves ATSP using the MTZ formulation.
    
    Args:
        location_list: A list of locations.
        distance_dict: A dict of dicts keyed by locations
            contatining all pairwise distances between the locations.
    
    Returns:
        Tour represented as a list of locations in the input location_list in optimal traversal order.
        To represent a closed tour, the starting location should be repeated as the last location in the list.
    '''
    # Write your implementation here
    return []

## Testing

Test your implementation by running the code below. Note that the full list of 21 locations is used here, so the computation may take a few seconds.

In [ ]:
tour = atsp_via_mtz(location_list, distance_dict)
print(f'Tour: {tour}')
print(f'Tour length: {sum(distance_dict[tour[i - 1]][tour[i]] for i in range(1, len(tour)))}')
visualize_mplleaflet_map(tour, coord_dict, 'map_full.html')

Note that the above cell only tests whether your implementation works - we are not testing it for correctness here. While one could easily implement functionality to test whether the returned location list is actually a tour (i.e., contains every input location precisely once), testing whether it is a shortest tour is just equally hard as computing a shortest tour. For the complete tour on all 21 locations above, you should reach a tour length of `59193.2`.

When testing, we can also consider a subset of locations to reduce the amount of time needed to solve the MTZ model. Here we provide an example where we choose a random sublist of locations to compute an ATSP tour for. Feel free to change the seed (or delete it to use a random one in every new execution), or explicitely change the sublist of locations to whatever you see fit to test your implementation. We provide expected tour length outputs for a few seeds below.

In [ ]:
import random

random.seed(0)

location_sublist = random.sample(location_list, random.randrange(3, len(location_list) + 1))
smaller_tour = atsp_via_mtz(location_sublist, distance_dict)
print(f'Tour: {smaller_tour}')
print(f'Tour length: {sum(distance_dict[smaller_tour[i - 1]][smaller_tour[i]] for i in range(1, len(smaller_tour)))}')
visualize_mplleaflet_map(smaller_tour, coord_dict, 'map_small.html')

In [ ]:
# Expected lengths for different seeds:
#
# seed | tour length
#   0  | 56110.200000000004
#   1  | 12609.000000000002
#   2  | 14904.400000000001
#   3  | 48085.5
#   4  | 38942.9
#   5  | 38088.6

## Measuring running time

Using the implementation from above and the tools seen earlier in the course, generate a plot that shows how your implementation of the ATSP solver via the MTZ formulation performs in terms of CPU time it takes to solve it depending on the instance size. To this end, you can for example generate a few random subsets of the 21 given locations for each size, measure the running time of the algorithm on that instance, and display the results in a scatter plot.

You should see clearly that the MTZ formulation is not the tool of first choice if instances get larger. We will cover other methods and techniques for ATSP later in the course that can help speeding up the process for larger instances.

In [ ]:
# implement your measurements here
